# Retail Customer Purchase Prediction
**COSC 4368 — Fundamentals of AI**

Team: Matthew Nguyen, Benjamin Tran, Victor Bui, Gustavo Buenrostro  
Date: (auto)

## Abstract (fill after results)
Brief problem, approach, and key result (ROC-AUC, F1).

## Acknowledgments
Instructor, peers; dataset: UCI Online Shoppers Intention (primary), UCI Online Retail II (optional RFM).


In [ ]:
# 1) Reproducibility & environment
import os, sys, json, time, math, numpy as np, pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix,
    roc_curve, auc, precision_recall_curve, classification_report, brier_score_loss
)
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

SEED = 42
np.random.seed(SEED)

# project paths (repo root → this notebook under notebooks/)
ROOT = Path().resolve().parents[0]  # Assumes notebook is in notebook/ subdirectory
DATA_DIR = ROOT / "data"
OUT_FIG_BASE = ROOT / "outputs" / "figures"
OUT_TAB_BASE = ROOT / "outputs" / "tables"
OUT_FIG_BASE.mkdir(parents=True, exist_ok=True)
OUT_TAB_BASE.mkdir(parents=True, exist_ok=True)

RUN_STAMP = time.strftime('%Y%m%d-%H%M%S')
OUT_FIG = OUT_FIG_BASE / RUN_STAMP
OUT_TAB = OUT_TAB_BASE / RUN_STAMP
OUT_FIG.mkdir(parents=True, exist_ok=True)
OUT_TAB.mkdir(parents=True, exist_ok=True)

print("Python:", sys.version)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("sklearn:", sklearn.__version__)
print("Save figs →", OUT_FIG)
print("Save tables →", OUT_TAB)
plt.rcParams["figure.dpi"] = 150


## 2) Data (primary) & feature plan
Primary: UCI Online Shoppers (`Revenue` is the label).  
Optional: Online Retail II → RFM enrichment later.


In [ ]:
# Load primary dataset
TARGET = "Revenue"
df = pd.read_csv(DATA_DIR / "online_shoppers_intention.csv")
if df[TARGET].dtype != int and df[TARGET].dtype != "int64":
    df[TARGET] = df[TARGET].astype(int)
df.shape, df.head()


In [ ]:
# Basic audit
audit = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_null": df.isnull().sum(),
    "n_unique": df.nunique()
}).sort_index()
audit.to_csv(OUT_TAB / "data_audit.csv")
audit.head(20)


**Feature groups (plan)**
- Behavioral: Administrative, Administrative_Duration, Informational, Informational_Duration,
  ProductRelated, ProductRelated_Duration, BounceRates, ExitRates, PageValues, SpecialDay  
- Temporal/Tech: Month, Weekend, OperatingSystems, Browser, Region, TrafficType, VisitorType  
- RFM (optional extension)


## 3) EDA & leakage audit


In [ ]:
# Class balance
counts = df[TARGET].value_counts().sort_index()
counts.to_csv(OUT_TAB / "class_counts.csv")
ax = counts.plot(kind="bar", title="Class Counts")
fig = ax.get_figure(); fig.tight_layout(); fig.savefig(OUT_FIG / "class_counts.png"); plt.close(fig)
counts, (counts / counts.sum()).rename("proportion")


In [ ]:
# Purchase rate by Month and VisitorType (if present)
if "Month" in df.columns:
    pr_month = df.groupby("Month")[TARGET].mean().sort_values()
    ax = pr_month.plot(kind="bar", title="Purchase Rate by Month")
    fig = ax.get_figure(); fig.tight_layout(); fig.savefig(OUT_FIG / "purchase_rate_by_month.png"); plt.close(fig)

if "VisitorType" in df.columns:
    pr_vtype = df.groupby("VisitorType")[TARGET].mean().sort_values()
    ax = pr_vtype.plot(kind="bar", title="Purchase Rate by VisitorType")
    fig = ax.get_figure(); fig.tight_layout(); fig.savefig(OUT_FIG / "purchase_rate_by_visitortype.png"); plt.close(fig)


**Leakage guardrails:** only use features known by session end; all transforms happen inside `Pipeline(pre, clf)` fit on **train only**.


## 4) Preprocessing & split


In [ ]:
# Identify numeric and categorical columns
NUM_COLS = df.drop(columns=[TARGET]).select_dtypes(include=["int64","float64","int32","float32"]).columns.tolist()
CAT_COLS = [c for c in df.drop(columns=[TARGET]).columns if c not in NUM_COLS]

pre = ColumnTransformer([
    ("num", StandardScaler(), NUM_COLS),
    ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_COLS)
])

# Split: 70/15/15 stratified
X = df.drop(columns=[TARGET])
y = df[TARGET].astype(int)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=SEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=SEED
)
len(X_train), len(X_val), len(X_test)


In [ ]:
# small utilities
from collections import OrderedDict

def fit_eval(pipe, name, Xtr=X_train, ytr=y_train, Xv=X_val, yv=y_val):
    pipe.fit(Xtr, ytr)
    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        y_score = pipe.predict_proba(Xv)[:,1]
    else:
        y_score = pipe.decision_function(Xv)
    y_pred = (y_score >= 0.5).astype(int)
    met = OrderedDict(
        model=name,
        roc_auc_val=roc_auc_score(yv, y_score),
        f1_val=f1_score(yv, y_pred),
        precision_val=precision_score(yv, y_pred, zero_division=0),
        recall_val=recall_score(yv, y_pred)
    )
    return pipe, met

val_rows = []
fitted = {}


## 5) Baselines (classical ML)


In [ ]:
# Logistic Regression (interpretable anchor)
lr_pipe = Pipeline([("pre", pre), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=SEED))])
lr_pipe, m = fit_eval(lr_pipe, "LogisticRegression")
fitted["LogisticRegression"] = lr_pipe; val_rows.append(m)
pd.DataFrame(val_rows)


In [ ]:
# Decision Tree (quick depth sweep)
best_dt, best_m_dt = None, None
for d in [3,5,10,None]:
    dt = Pipeline([("pre", pre), ("clf", DecisionTreeClassifier(max_depth=d, random_state=SEED))])
    dt, mdict = fit_eval(dt, f"DecisionTree(max_depth={d})")
    if best_m_dt is None or mdict["roc_auc_val"] > best_m_dt["roc_auc_val"]:
        best_dt, best_m_dt = dt, mdict
fitted["DecisionTree"] = best_dt; val_rows.append(best_m_dt)
val_df = pd.DataFrame(val_rows).sort_values("roc_auc_val", ascending=False)
val_df.to_csv(OUT_TAB / "validation_metrics.csv", index=False)
val_df


In [ ]:
# Validation confusion matrix for LR
pipe = fitted["LogisticRegression"]
if hasattr(pipe.named_steps["clf"], "predict_proba"):
    y_score = pipe.predict_proba(X_val)[:,1]
else:
    y_score = pipe.decision_function(X_val)
y_pred = (y_score >= 0.5).astype(int)
cm = confusion_matrix(y_val, y_pred)
fig = plt.figure(); ax = fig.add_subplot(111)
ax.imshow(cm, interpolation="nearest")
ax.set_title("Validation Confusion Matrix — LogisticRegression")
ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
for (i,j), v in np.ndenumerate(cm):
    ax.text(j, i, str(v), ha="center", va="center")
fig.tight_layout(); fig.savefig(OUT_FIG / "confusion_matrix_val_logreg.png"); plt.close(fig)


In [ ]:
# Top LR coefficients (absolute) using feature names from fitted preprocessor
pre_fit = pre.fit(X_train, y_train)
try:
    cat_names = list(pre_fit.named_transformers_["cat"].get_feature_names_out(CAT_COLS))
except Exception:
    cat_names = [f"{c}_{i}" for i, c in enumerate(CAT_COLS)]
feat_names = list(NUM_COLS) + cat_names

clf = fitted["LogisticRegression"].named_steps["clf"]
if hasattr(clf, "coef_"):
    coefs = np.ravel(clf.coef_)
    order = np.argsort(np.abs(coefs))[::-1][:25]
    fig = plt.figure(figsize=(6,8)); ax = fig.add_subplot(111)
    ax.barh(range(len(order)), np.abs(coefs[order])[::-1])
    ax.set_yticks(range(len(order))); ax.set_yticklabels(np.array(feat_names)[order][::-1])
    ax.set_title("Top |Coefficient| — Logistic Regression"); ax.set_xlabel("Absolute Weight")
    fig.tight_layout(); fig.savefig(OUT_FIG / "lr_top_coefs.png"); plt.close(fig)


## 6) Ensembles


In [ ]:
# Random Forest (coarse CV grid)
rf = RandomForestClassifier(random_state=SEED, n_estimators=300, n_jobs=-1)
rf_pipe = Pipeline([("pre", pre), ("clf", rf)])
param_grid = {
    "clf__max_depth": [None, 10, 20],
    "clf__min_samples_split": [2, 5, 10]
}
rf_cv = GridSearchCV(rf_pipe, param_grid, scoring="roc_auc", cv=5, n_jobs=-1)
rf_cv.fit(X_train, y_train)
rf_best = rf_cv.best_estimator_

if hasattr(rf_best.named_steps["clf"], "predict_proba"):
    y_score = rf_best.predict_proba(X_val)[:,1]
else:
    y_score = rf_best.decision_function(X_val)
y_pred = (y_score >= 0.5).astype(int)
m = dict(model="RandomForest", roc_auc_val=roc_auc_score(y_val, y_score),
         f1_val=f1_score(y_val, y_pred), precision_val=precision_score(y_val, y_pred, zero_division=0),
         recall_val=recall_score(y_val, y_pred))
fitted["RandomForest"] = rf_best; val_rows.append(m)

# Feature importance
clf = rf_best.named_steps["clf"]
if hasattr(clf, "feature_importances_"):
    imps = np.asarray(clf.feature_importances_)
    order = np.argsort(imps)[::-1][:20]
    fig = plt.figure(figsize=(6,8)); ax = fig.add_subplot(111)
    ax.barh(range(len(order)), imps[order][::-1])
    ax.set_yticks(range(len(order))); ax.set_yticklabels(np.array(feat_names)[order][::-1])
    ax.set_title("Top Feature Importance — Random Forest"); ax.set_xlabel("Importance")
    fig.tight_layout(); fig.savefig(OUT_FIG / "feature_importance_top20.png"); plt.close(fig)

val_df = pd.DataFrame(val_rows).sort_values("roc_auc_val", ascending=False)
val_df.to_csv(OUT_TAB / "validation_metrics.csv", index=False)
val_df


In [ ]:
# (Optional) Gradient Boosting — enable if course allows
try:
    gb = GradientBoostingClassifier(random_state=SEED)
    gb_pipe = Pipeline([("pre", pre), ("clf", gb)])
    gb_params = {"clf__learning_rate":[0.05,0.1], "clf__n_estimators":[100,200], "clf__max_depth":[3]}
    gb_cv = GridSearchCV(gb_pipe, gb_params, scoring="roc_auc", cv=5, n_jobs=-1)
    gb_cv.fit(X_train, y_train)
    gb_best = gb_cv.best_estimator_
    if hasattr(gb_best.named_steps["clf"], "predict_proba"):
        y_score = gb_best.predict_proba(X_val)[:,1]
    else:
        y_score = gb_best.decision_function(X_val)
    y_pred = (y_score >= 0.5).astype(int)
    m = dict(model="GradientBoosting", roc_auc_val=roc_auc_score(y_val, y_score),
             f1_val=f1_score(y_val, y_pred), precision_val=precision_score(y_val, y_pred, zero_division=0),
             recall_val=recall_score(y_val, y_pred))
    fitted["GradientBoosting"] = gb_best; val_rows.append(m)
except Exception as e:
    print("GradientBoosting skipped:", e)

val_df = pd.DataFrame(val_rows).sort_values("roc_auc_val", ascending=False)
val_df.to_csv(OUT_TAB / "validation_metrics.csv", index=False)
val_df


## 7) Neural network (shallow MLP)


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(64,), activation="relu", alpha=1e-4,
                    early_stopping=True, random_state=SEED, max_iter=200)
mlp_pipe = Pipeline([("pre", pre), ("clf", mlp)])
mlp_pipe, m = fit_eval(mlp_pipe, "MLP(64)")
fitted["MLP"] = mlp_pipe; val_rows.append(m)
val_df = pd.DataFrame(val_rows).sort_values("roc_auc_val", ascending=False)
val_df.to_csv(OUT_TAB / "validation_metrics.csv", index=False)
val_df


In [ ]:
# Learning curve (5-fold CV) for MLP
sizes, train_scores, val_scores = learning_curve(mlp_pipe, X_train, y_train, cv=5, scoring="roc_auc",
                                                 n_jobs=-1, train_sizes=np.linspace(0.2,1.0,5), shuffle=True, random_state=SEED)
tr = train_scores.mean(axis=1); te = val_scores.mean(axis=1)
fig = plt.figure(); ax = fig.add_subplot(111)
ax.plot(sizes, tr, marker="o", label="Train")
ax.plot(sizes, te, marker="o", label="CV")
ax.set_title("Learning Curve — MLP"); ax.set_xlabel("Train size"); ax.set_ylabel("ROC-AUC"); ax.legend(loc="best")
fig.tight_layout(); fig.savefig(OUT_FIG / "mlp_learning_curve.png"); plt.close(fig)


In [ ]:
# Pick best model by validation ROC-AUC
val_df = pd.DataFrame(val_rows).sort_values("roc_auc_val", ascending=False)
best_name = val_df.iloc[0]["model"]
best_pipe = fitted[best_name]

# validation scores
if hasattr(best_pipe.named_steps["clf"], "predict_proba"):
    y_score_val = best_pipe.predict_proba(X_val)[:,1]
else:
    y_score_val = best_pipe.decision_function(X_val)

ths = np.linspace(0.1, 0.9, 33)
rows = []
for t in ths:
    pred = (y_score_val >= t).astype(int)
    rows.append({
        "threshold": t,
        "f1": f1_score(y_val, pred),
        "precision": precision_score(y_val, pred, zero_division=0),
        "recall": recall_score(y_val, pred)
    })
thr_df = pd.DataFrame(rows)
thr_df.to_csv(OUT_TAB / "threshold_sweep_val.csv", index=False)

fig = plt.figure(); ax = fig.add_subplot(111)
ax.plot(thr_df["threshold"], thr_df["precision"], label="Precision")
ax.plot(thr_df["threshold"], thr_df["recall"], label="Recall")
ax.plot(thr_df["threshold"], thr_df["f1"], label="F1")
ax.set_xlabel("Threshold"); ax.set_ylabel("Score")
ax.set_title(f"Threshold Sweep — {best_name} (Validation)"); ax.legend(loc="best")
fig.tight_layout(); fig.savefig(OUT_FIG / "threshold_sweep.png"); plt.close(fig)

best_thr = float(thr_df.sort_values("f1", ascending=False).iloc[0]["threshold"])
(Path(OUT_TAB / "chosen_threshold.json")).write_text(json.dumps({"model": best_name, "threshold": best_thr}, indent=2))
best_name, best_thr


## 9) Test evaluation & calibration


In [ ]:
best_name = json.loads((OUT_TAB / "chosen_threshold.json").read_text())["model"]
best_thr = json.loads((OUT_TAB / "chosen_threshold.json").read_text())["threshold"]
pipe = fitted[best_name]

# Scores
if hasattr(pipe.named_steps["clf"], "predict_proba"):
    y_score_test = pipe.predict_proba(X_test)[:,1]
else:
    y_score_test = pipe.decision_function(X_test)

y_pred_test = (y_score_test >= best_thr).astype(int)

test_metrics = {
    "model": best_name,
    "threshold": best_thr,
    "roc_auc_test": roc_auc_score(y_test, y_score_test),
    "f1_test": f1_score(y_test, y_pred_test),
    "precision_test": precision_score(y_test, y_pred_test, zero_division=0),
    "recall_test": recall_score(y_test, y_pred_test)
}
pd.DataFrame([test_metrics]).to_csv(OUT_TAB / "test_metrics.csv", index=False)
print(test_metrics)

# ROC
fpr, tpr, _ = roc_curve(y_test, y_score_test)
fig = plt.figure(); ax = fig.add_subplot(111)
ax.plot(fpr, tpr, label=f"AUC={auc(fpr,tpr):.3f}")
ax.plot([0,1],[0,1],"--")
ax.set_title(f"ROC Curve — Test ({best_name})"); ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.legend(loc="lower right")
fig.tight_layout(); fig.savefig(OUT_FIG / "roc_curve_test.png"); plt.close(fig)

# PR
prec, rec, _ = precision_recall_curve(y_test, y_score_test)
fig = plt.figure(); ax = fig.add_subplot(111)
ax.plot(rec, prec)
ax.set_title("Precision–Recall Curve — Test"); ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
fig.tight_layout(); fig.savefig(OUT_FIG / "pr_curve_test.png"); plt.close(fig)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
fig = plt.figure(); ax = fig.add_subplot(111)
ax.imshow(cm, interpolation="nearest")
ax.set_title("Confusion Matrix — Test"); ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
for (i,j), v in np.ndenumerate(cm):
    ax.text(j, i, str(v), ha="center", va="center")
fig.tight_layout(); fig.savefig(OUT_FIG / "confusion_matrix_test.png"); plt.close(fig)

# Calibration / Reliability & Brier
prob_true, prob_pred = calibration_curve(y_test, y_score_test, n_bins=10, strategy="quantile")
fig = plt.figure(); ax = fig.add_subplot(111)
ax.plot(prob_pred, prob_true, marker="o"); ax.plot([0,1],[0,1],"--")
ax.set_title("Reliability (Calibration) — Test"); ax.set_xlabel("Predicted probability"); ax.set_ylabel("Observed frequency")
brier = brier_score_loss(y_test, y_score_test); ax.text(0.6, 0.1, f"Brier={brier:.3f}", transform=ax.transAxes)
fig.tight_layout(); fig.savefig(OUT_FIG / "calibration_curve_test.png"); plt.close(fig)
(OUT_TAB / "brier_score.txt").write_text(str(brier))


## 10) Interpretability


In [ ]:
# Expanded feature names from fitted preprocessor
pre_fit = pre.fit(X_train, y_train)
try:
    cat_names = list(pre_fit.named_transformers_["cat"].get_feature_names_out(CAT_COLS))
except Exception:
    cat_names = [f"{c}_{i}" for i, c in enumerate(CAT_COLS)]
feat_names = np.array(list(NUM_COLS) + cat_names)

pipe = fitted[best_name]
clf = pipe.named_steps["clf"]
vals = None; label = "Importance"
if hasattr(clf, "feature_importances_"):
    vals = np.asarray(clf.feature_importances_); label = "Feature Importance"
elif hasattr(clf, "coef_"):
    vals = np.abs(np.ravel(clf.coef_)); label = "|Coefficient|"

if vals is not None:
    order = np.argsort(vals)[::-1][:25]
    fig = plt.figure(figsize=(6,8)); ax = fig.add_subplot(111)
    ax.barh(range(len(order)), vals[order][::-1])
    ax.set_yticks(range(len(order))); ax.set_yticklabels(feat_names[order][::-1])
    ax.set_title(f"Top {label} — {best_name}"); ax.set_xlabel(label)
    fig.tight_layout(); fig.savefig(OUT_FIG / "feature_importance_top25.png"); plt.close(fig)
else:
    print("Model does not expose feature importance or coefficients.")


## 11) Ablations


In [ ]:
BEHAVIORAL = [c for c in [
    "Administrative","Administrative_Duration","Informational","Informational_Duration",
    "ProductRelated","ProductRelated_Duration","BounceRates","ExitRates","PageValues","SpecialDay"
] if c in X.columns]

TEMPORAL_TECH = [c for c in [
    "Month","Weekend","OperatingSystems","Browser","Region","TrafficType","VisitorType"
] if c in X.columns]

def make_pre(cols_keep):
    num = [c for c in cols_keep if c in NUM_COLS]
    cat = [c for c in cols_keep if c in CAT_COLS]
    return ColumnTransformer([("num", StandardScaler(), num), ("cat", OneHotEncoder(handle_unknown="ignore"), cat)])

def eval_group(cols_keep, label):
    pre_g = make_pre(cols_keep)
    # reuse best model family for fairness
    if best_name.startswith("RandomForest"):
        clf = RandomForestClassifier(random_state=SEED, n_estimators=300, n_jobs=-1)
    elif best_name.startswith("GradientBoosting") or best_name == "GradientBoosting":
        clf = GradientBoostingClassifier(random_state=SEED)
    elif best_name.startswith("MLP") or best_name == "MLP(64)":
        clf = MLPClassifier(hidden_layer_sizes=(64,), activation="relu", alpha=1e-4, early_stopping=True, random_state=SEED, max_iter=200)
    elif best_name.startswith("Logistic") or best_name == "LogisticRegression":
        clf = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=SEED)
    else:
        clf = RandomForestClassifier(random_state=SEED, n_estimators=300, n_jobs=-1)
    pipe = Pipeline([("pre", pre_g), ("clf", clf)])
    _, m = fit_eval(pipe, f"{best_name} | {label}")
    return m

abl_rows = []
# Full
full_row = [r for r in val_rows if r["model"]==best_name]
if full_row:
    abl_rows.append({"group":"Full", "roc_auc_val": full_row[0]["roc_auc_val"]})

if BEHAVIORAL:
    m = eval_group(BEHAVIORAL, "BehavioralOnly")
    abl_rows.append({"group":"BehavioralOnly", "roc_auc_val": m["roc_auc_val"]})

if TEMPORAL_TECH:
    m = eval_group(TEMPORAL_TECH, "TemporalTechOnly")
    abl_rows.append({"group":"TemporalTechOnly", "roc_auc_val": m["roc_auc_val"]})

abl_df = pd.DataFrame(abl_rows)
abl_df.to_csv(OUT_TAB / "ablations.csv", index=False)

if "Full" in abl_df["group"].values:
    base = float(abl_df.loc[abl_df["group"]=="Full","roc_auc_val"])
    abl_df2 = abl_df.copy()
    abl_df2["delta_auc"] = abl_df2["roc_auc_val"] - base
    ax = abl_df2.set_index("group")["delta_auc"].plot(kind="bar", title="Ablation ΔAUC vs Full")
    fig = ax.get_figure(); fig.tight_layout(); fig.savefig(OUT_FIG / "ablation_delta_auc.png"); plt.close(fig)
abl_df


## 12) Business takeaways (fill 5–7 bullets)
- Signals that most increase purchase likelihood  
- Recommended operating threshold and trade-offs  
- Implications for remarketing and UX


## 13) Conclusion & future direction
- Best model and trade-offs  
- Future: calibration tuning, cost-sensitive metrics, RFM enrichment, deployment hooks


## 14) Optional RFM (Online Retail II)
Hook for building Recency, Frequency, Monetary features and joining if a key exists.


In [ ]:
# 15) Export helpers & final assets
def _save(fig, name):
    path = (OUT_FIG / f"{name}.png")
    fig.savefig(path, dpi=300, bbox_inches="tight"); plt.close(fig); return str(path)

def save_confusion_matrix(model, X, y, name):
    y_hat = model.predict(X)
    cm = confusion_matrix(y, y_hat)
    fig = plt.figure(); ax = fig.add_subplot(111)
    ax.imshow(cm, interpolation="nearest"); ax.set_title("Confusion Matrix")
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    for (i,j), v in np.ndenumerate(cm): ax.text(j, i, str(v), ha="center", va="center")
    return _save(fig, name)

def save_roc(model, X, y, name):
    if hasattr(model.named_steps["clf"], "predict_proba"): y_score = model.predict_proba(X)[:,1]
    else: y_score = model.decision_function(X)
    fpr, tpr, _ = roc_curve(y, y_score)
    fig = plt.figure(); ax = fig.add_subplot(111)
    ax.plot(fpr, tpr, label=f"AUC={auc(fpr,tpr):.3f}"); ax.plot([0,1],[0,1],"--")
    ax.set_title("ROC Curve"); ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.legend(loc="lower right")
    return _save(fig, name)

def save_pr(model, X, y, name):
    if hasattr(model.named_steps["clf"], "predict_proba"): y_score = model.predict_proba(X)[:,1]
    else: y_score = model.decision_function(X)
    precision, recall, _ = precision_recall_curve(y, y_score)
    fig = plt.figure(); ax = fig.add_subplot(111)
    ax.plot(recall, precision); ax.set_title("Precision–Recall Curve"); ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    return _save(fig, name)

def save_calibration(model, X, y, name):
    if hasattr(model.named_steps["clf"], "predict_proba"): y_score = model.predict_proba(X)[:,1]
    else:
        y_score = model.decision_function(X)
        y_score = (y_score - y_score.min()) / (y_score.max() - y_score.min() + 1e-12)
    prob_true, prob_pred = calibration_curve(y, y_score, n_bins=10, strategy="quantile")
    fig = plt.figure(); ax = fig.add_subplot(111)
    ax.plot(prob_pred, prob_true, marker="o"); ax.plot([0,1],[0,1],"--")
    ax.set_title("Reliability (Calibration)"); ax.set_xlabel("Predicted probability"); ax.set_ylabel("Observed frequency")
    brier = brier_score_loss(y, y_score); ax.text(0.6, 0.1, f"Brier={brier:.3f}", transform=ax.transAxes)
    return _save(fig, name)

def save_feature_importance_fig(model, feat_names, name):
    clf = model.named_steps["clf"]
    vals = None; label="Importance"
    if hasattr(clf, "feature_importances_"):
        vals = np.asarray(clf.feature_importances_); label="Feature Importance"
    elif hasattr(clf, "coef_"):
        vals = np.abs(np.ravel(clf.coef_)); label="|Coefficient|"
    if vals is None: return None
    order = np.argsort(vals)[::-1][:25]
    fig = plt.figure(figsize=(6,8)); ax = fig.add_subplot(111)
    ax.barh(range(len(order)), vals[order][::-1])
    ax.set_yticks(range(len(order))); ax.set_yticklabels(np.array(feat_names)[order][::-1])
    ax.set_title(f"Top {label}"); ax.set_xlabel(label)
    return _save(fig, name)

def export_all(model, X_test, y_test, feat_names, prefix="purchase_pred"):
    paths = {}
    paths["roc"] = save_roc(model, X_test, y_test, f"{prefix}_roc_test")
    paths["pr"] = save_pr(model, X_test, y_test, f"{prefix}_pr_test")
    paths["cm"] = save_confusion_matrix(model, X_test, y_test, f"{prefix}_cm_test")
    paths["cal"] = save_calibration(model, X_test, y_test, f"{prefix}_calibration_test")
    fi = save_feature_importance_fig(model, feat_names, f"{prefix}_feature_importance")
    if fi: paths["feature_importance"] = fi
    rep = classification_report(y_test, model.predict(X_test), output_dict=True)
    (OUT_TAB / f"{prefix}_classification_report.json").write_text(json.dumps(rep, indent=2))
    return paths

# Expanded names for feature-importance figure
pre_fit = pre.fit(X_train, y_train)
try:
    cat_names = list(pre_fit.named_transformers_["cat"].get_feature_names_out(CAT_COLS))
except Exception:
    cat_names = [f"{c}_{i}" for i, c in enumerate(CAT_COLS)]
feat_names = list(NUM_COLS) + cat_names

export_paths = export_all(fitted[best_name], X_test, y_test, feat_names, prefix="purchase_pred")
(OUT_TAB / "export_paths.json").write_text(json.dumps(export_paths, indent=2, ensure_ascii=False))

# Collect a curated set for slides/poster
ASSET_DIR = ROOT / "presentations" / "assets"
ASSET_DIR.mkdir(parents=True, exist_ok=True)
ordered = ["class_counts.png","threshold_sweep.png","roc_curve_test.png","confusion_matrix_test.png",
           "feature_importance_top20.png","feature_importance_top25.png","mlp_learning_curve.png","ablation_delta_auc.png"]
i = 1
for name in ordered:
    src = OUT_FIG / name
    if src.exists():
        (ASSET_DIR / f"{i:02d}_{name}").write_bytes(src.read_bytes()); i += 1
print("Assets exported →", ASSET_DIR)
